# Capstone Project - The Battle of Neighborhoods (Week 1)

## 1. A description of the problem and a discussion of the background. 

### Introduction of the City - Los Angeles

Los Angeles, officially the City of Los Angeles and often known by its initials L.A., is the largest city in California. With an estimated population of nearly four million people, it is the second-most populous city in the United States (after New York City) and the third-most populous city in North America (after Mexico City and New York City). Los Angeles is known for its Mediterranean climate, ethnic diversity, Hollywood entertainment industry, and its sprawling metropolis. Los Angeles lies in a basin in Southern California, adjacent to the Pacific Ocean, with mountains as high as 10,000 feet (3,000 m), and deserts. The city, which covers about 469 square miles (1,210 km2),is the seat of Los Angeles County, the most populous county in the United States. The Los Angeles metropolitan area (MSA) is home to 13.1 million people, making it the second-largest metropolitan area in the nation after New York. Greater Los Angeles includes metro Los Angeles as well as the Inland Empire and Ventura County.It is the second-most populous U.S. combined statistical area, also after New York, with a 2015 estimate of 18.7 million people.

### Business Problem

Coming down to business problem,I would like to open a restaurant near Santa Monica, which is one of the most beautiful cities in Los Angeles. As it is a famous tourist spot,there is already lots of attention towards it.I know there will be many competitors in terms of restauramt. But keeping them in mind, I need to locate my restaurant in place where more people are attracted and comfortable for a stay and a good meal.I want to bring foreign and local peoples attention towards my new hotel.I would like to flavour my retaurant recipe with Chinese and Japanese to grab their taste.
The challenge is to find a suitable location for opening a new restaurant attracted to all local and foreign people in the centre of all famous venues.

### Assumption

I assume that 85% local and 15% foreign peoples visit Santa Monica once in a year. Some people stay for couple of days or more. Also they find some place for hangout or a good meal. Their main focus might be belonging to stay somewhere near to reach venues. Apart from these set of people, students and working professionals are common audience here. So we may need to fascinate them all.

# Data Selection

We will be completely working on Foursquare data to explore and try to locate our new restaurant where more venues like church, temples, beach, museums, memorials that are present nearby.

We will looking for midpoint area of venues to locate our new restaurant. Before that our major focus will be on all venues present in and around the core place of Santa Monica.

Just a heads up on how many restaurant are distributed now around Santa Monica. We will perform some EDA on restaurants present in the tourist spot. On furthur notebook we will use Foursquare data to determine other venues as well.

# Data Preprocessing

### Import Libraries

In [2]:
# Import libraries
import numpy as np # data in a vectorized manner manipulation
import pandas as pd # data analsysis
import requests # HTTP library
from bs4 import BeautifulSoup # scraping library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # JSON files manipulation

from sklearn.cluster import KMeans # clustering algorithm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("Libraries Imported")

Libraries Imported


### Credentials and Core location

In [3]:
CLIENT_ID = '3Z3ZIFG1GY2NGYXZ3AVE0DZKPAVVCQJHTENE4IR5OXJNTNPW' 
CLIENT_SECRET = 'J2WB5IYMG4AJCWMVF0ABC5SR1YRH3JOEQ2HOLBOGJE0TF5DB '
VERSION = '20200811'
LIMIT = 150

address = "Santa Monica, Los Angeles"

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude =location.latitude    
longitude =location.longitude  

SM='Santa Monica location : {},{}'.format(latitude,longitude)
print(SM)

Santa Monica location : 34.0250724,-118.4965129


### Search Restaurants within 1 km

In [4]:
search_query_res = 'restaurant'
radius = 1000
url_restaurant = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query_res, radius, LIMIT)

### Send the Get Request of Restaurants and Examine the Results

In [6]:
results_restaurant = requests.get(url_restaurant).json()

### Get Relevant Part of JSON and Transform it into a Pandas Dataframe

In [8]:
# assign relevant part of JSON to venues
venues_restaurant = results_restaurant['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues_restaurant)

print("There are {} restaurants at Santa Monica".format(dataframe.shape[0]))

There are 16 restaurants at Santa Monica


### Define Information of Interest and Filter Dataframe

In [9]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

    
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

  
# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#dataframe_filtered
Restaurants_df=dataframe_filtered[['name','categories','distance','lat','lng','id']]
Restaurants_df.head()

,name,categories,distance,lat,lng,id
0,The Misfit Restaurant + Bar,American Restaurant,1076,34.015399,-118.496591,4da4bb1d0f578cfaafdbd518
1,Michael's Restaurant,American Restaurant,766,34.018740,-118.499770,4acfed0ff964a520e6d620e3
2,Restaurant chino Nan hai,Chinese Restaurant,795,34.019454,-118.491191,503a8c8ee4b0bbf1b03b7e1e
3,Simple Things Restaurant,American Restaurant,393,34.022205,-118.494011,5635381d498e8f8d6a6e9bf2
4,Romano's Restaurant,Food,520,34.026932,-118.501684,4f321f6719836c91c7b83bac


### Location of Restaurants

In [11]:
Restaurants_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around the Santa Monica

# add a red circle marker to represent the core location of Santa Monica
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Santa Monica',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(Restaurants_map)

# add the restaurants as blue circle markers
for lat, lng, label in zip(Restaurants_df.lat, Restaurants_df.lng, Restaurants_df.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(Restaurants_map)

# display map
Restaurants_map

Here are all contents for the Capstone Project - The Battle of Neighborhoods (Week 1), I will cover my code and report in the next Notebook. Thanks!